In [1]:
%load_ext autoreload
%autoreload 2
import time
from datetime import datetime, date, time, timedelta
import asyncio
import nest_asyncio
import importlib
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
from tastytrade import Account, Session, DXLinkStreamer
from tastytrade.dxfeed import Quote, Candle
from tastytrade.instruments import *
from tastytrade.market_data import *
from config import *
from alphas import *
from itertools import chain

nest_asyncio.apply()

In [2]:
session=None
account=None

async def setup():
    global session, account
    # setup
    config = Config(test=False)
    session = Session(config.username, config.password, is_test=config.test)
    account = await Account.a_get(session, config.account_number)

    # account info
    balance = account.get_balances(session)
    positions = account.get_positions(session)
    history = account.get_history(session)

asyncio.run(setup())

In [3]:
streamer = DXLinkStreamer(session) #async streamer

SPXoptionchain = get_option_chain(session, "SPX")
VIXoptionchain = get_option_chain(session, "VIX")

SPXexpiries = list(SPXoptionchain.keys())
VIXexpiries = list(VIXoptionchain.keys())
#print(SPXexpiries)
VXfutures = Future.get(session, symbols = None, product_codes=["VX"])
VXfm = VXfutures[-2]
VXbm = VXfutures[-1]
VXfmdata = get_market_data_by_type(session,futures=[VXfm.symbol])
VXbmdata = get_market_data_by_type(session,futures=[VXbm.symbol])
#print(VXfmdata)
indexreq = ["SPX"]
SPXdata = get_market_data_by_type(session,indices = indexreq)
#print(type(SPXdata[0].last))

In [4]:
def calc_forward_iv(chain, dte, rfr, spot):  # need specifically the chain of the day
    T = dte / 365
    r = rfr / 100
    forward = spot * np.exp(r * T)
    chain = [opt for opt in chain if opt.settlement_type == "PM"]
    chain = convertchain(chain)

    K0 = chain.loc[abs(chain["strike"] - forward).idxmin(), "strike"]

    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]
    call_bound = otmcalls[
        ((otmcalls["cbid"] == 0) & (otmcalls.shift(-1)["cbid"] == 0))
        | ((otmcalls["cask"] == 0) & (otmcalls.shift(-1)["cask"] == 0))
        | (otmcalls["cbid"].isna())
        | (otmcalls["cask"].isna())
    ].index.min()
    put_bound = otmputs[
        ((otmputs["pbid"] == 0) & (otmputs.shift(1)["pbid"] == 0))
        | ((otmputs["pask"] == 0) & (otmputs.shift(1)["pask"] == 0))
        | (otmputs["pbid"].isna())
        | (otmputs["pask"].isna())
    ].index.max()

    if pd.notna(call_bound):
        chain = chain.loc[: call_bound - 1]  # right end is inclusive in pd loc
    if pd.notna(put_bound):
        chain = chain.loc[put_bound + 1 :]
    if otmcalls.empty or otmputs.empty:
        raise ValueError(
            f"no OTM options check code for {dte} dte"
        )

    chain = chain[
        ~(
            (
                (chain["strike"] >= K0)
                & ((chain["cbid"] == 0) | (chain["cask"] == 0))
            )
            | (
                (chain["strike"] <= K0)
                & ((chain["pbid"] == 0) | (chain["pask"] == 0))
            )
        )
    ]  # ignoring rows that have either 0 bid or ask

    chain["dK"] = (chain["strike"].shift(-1) - chain["strike"].shift(1)) / 2
    chain.loc[chain.index[0], "dK"] = (
        chain.loc[chain.index[1], "strike"] - chain.loc[chain.index[0], "strike"]
    )
    chain.loc[chain.index[-1], "dK"] = (
        chain.loc[chain.index[-1], "strike"]
        - chain.loc[chain.index[-2], "strike"]
    )
    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]

    call_vals = (
        (
            otmcalls[otmcalls["strike"] != K0]["cbid"]
            + otmcalls[otmcalls["strike"] != K0]["cask"]
        )
        / 2
        * (
            otmcalls[otmcalls["strike"] != K0]["dK"]
            / otmcalls[otmcalls["strike"] != K0]["strike"] ** 2
        )
    )
    put_vals = (
        (
            otmputs[otmputs["strike"] != K0]["pbid"]
            + otmputs[otmputs["strike"] != K0]["pask"]
        )
        / 2
        * (
            otmputs[otmputs["strike"] != K0]["dK"]
            / otmputs[otmputs["strike"] != K0]["strike"] ** 2
        )
    )
    forwardiv = (
        2
        / T
        * np.exp(r * T)
        * (
            np.sum(call_vals)
            + interpolate(otmcalls, "call", "fiv")
            + np.sum(put_vals)
            + interpolate(otmputs, "put", "fiv")
            + (otmcalls.iloc[0]["cbid"] + otmcalls.iloc[0]["cask"])
            / 4
            * (otmcalls.iloc[0]["dK"] / otmcalls.iloc[0]["strike"] ** 2)
            + (otmputs.iloc[-1]["pbid"] + otmputs.iloc[-1]["pask"])
            / 4
            * (otmputs.iloc[-1]["dK"] / otmputs.iloc[-1]["strike"] ** 2)
        )
        - 1 / T * (forward / K0 - 1) ** 2
    )

    return (
        10000 * forwardiv
    )  # 100^2, since varq is measured on vix options, therefore this also needs to be scaled to actual vix


def calc_varq(chain, dte, rfr, forward):
    # in varq the chain is the vix chain and spot is spot vix
    calls_bounded, puts_bounded = False, False
    T = dte / 365
    r = rfr / 100
    chain = convertchain(chain)
    chain["dK"] = (chain["strike"].shift(-1) - chain["strike"].shift(1)) / 2
    chain.loc[chain.index[0], "dK"] = (
        chain.loc[chain.index[1], "strike"] - chain.loc[chain.index[0], "strike"]
    )
    chain.loc[chain.index[-1], "dK"] = (
        chain.loc[chain.index[-1], "strike"]
        - chain.loc[chain.index[-2], "strike"]
    )
    K0 = chain.loc[abs(chain["strike"] - forward).idxmin(), "strike"]
    otmcalls = chain[chain["strike"] >= K0]
    otmputs = chain[chain["strike"] <= K0]
    call_bound = otmcalls[
        (otmcalls["cbid"] == 0)
        | (otmcalls["cask"] == 0)
        | (otmcalls["cbid"].isna())
        | (otmcalls["cask"].isna())
    ].index.min()
    put_bound = otmputs[
        (otmputs["pbid"] == 0)
        | (otmputs["pask"] == 0)
        | (otmputs["pbid"].isna())
        | (otmputs["pask"].isna())
    ].index.max()
    if pd.notna(call_bound):
        otmcalls = otmcalls.loc[: call_bound - 1]
        calls_bounded = True
    if pd.notna(put_bound):
        otmputs = otmputs.loc[put_bound + 1 :]
        puts_bounded = True
    if calls_bounded == False:
        print("vix calls unbounded")
    if puts_bounded == False:
        print("vix puts unbounded")

    if otmcalls.empty and otmputs.empty:
        raise ValueError(
            f"no OTM options on VIX check code for {dte} dte"
        )
    elif otmputs.empty and not otmcalls.empty:
        # print(f"no OTM puts on VIX on {chain.loc[:,' [QUOTE_DATE]'].iloc[0]} exp {chain.loc[:,' [EXPIRE_DATE]'].iloc[0]}")
        call_vals = (otmcalls["cbid"] + otmcalls["cask"]) / 2 * otmcalls["dK"]
        varq = 2 * np.exp(r * T) * (np.sum(call_vals)+(interpolate(otmcalls, "call", "varq") if not calls_bounded else 0))
    elif otmcalls.empty and not otmputs.empty:
        put_vals = (otmputs["pbid"] + otmputs["pask"]) / 2 * otmputs["dK"]
        varq = 2 * np.exp(r * T) * (np.sum(put_vals)+(interpolate(otmputs, "put", "varq") if not puts_bounded else 0))
    else:
        call_vals = (
            (
                otmcalls[otmcalls["strike"] != K0]["cbid"]
                + otmcalls[otmcalls["strike"] != K0]["cask"]
            )
            / 2
            * otmcalls[otmcalls["strike"] != K0]["dK"]
        )
        put_vals = (
            (
                otmputs[otmputs["strike"] != K0]["pbid"]
                + otmputs[otmputs["strike"] != K0]["pask"]
            )
            / 2
            * otmputs[otmputs["strike"] != K0]["dK"]
        )
        varq = (
            2
            * np.exp(r * T)
            * (
                np.sum(call_vals)
                + np.sum(put_vals)
                +(interpolate(otmcalls, "call", "varq") if not calls_bounded else 0)
                +(interpolate(otmputs, "put", "varq") if not puts_bounded else 0)
                + (otmcalls.iloc[0]["cbid"] + otmcalls.iloc[0]["cask"])
                / 4
                * otmcalls.iloc[0]["dK"]
                + (otmputs.iloc[-1]["pbid"] + otmputs.iloc[-1]["pask"])
                / 4
                * otmputs.iloc[-1]["dK"]
            )
        )
    return varq

def interpolate(chain, type, component):
    # sometimes, the chain gets "cut off" because there are not enough strikes. this results in an incorrect calculation of the implied vix future value
    # there are no easy workarounds. here are the few things we can do:
    # 1. ignore expiry if we cannot find call_bound or put_bound. however sometimes we cannot simply skip an expiry day (sometimes we do not have the luxury of finding another expiry day that works for our vx expiry)
    # 2. choosing only monthlies, but this is considerably less accurate, and in older years sometimes even monthlies do not have reliable strikes
    # 3. interpolating otm option prices, but this introduces errors due to assumptions being used in the distribution of prices. however, this is the most reasonable approach as of now
    # this interpolation function will attempt to estimate the size of the tail that is "cut off" from the option chain, and return 0 if the tail is not cut off (ie bound exists)

    # print(f'interpolating {chain.loc[:," [QUOTE_DATE]"].iloc[0]} exp {chain.loc[:," [EXPIRE_DATE]"].iloc[0]}')
    if len(chain) == 0:
        raise ValueError(
            f"interpolating null chain"
        )
    elif len(chain) == 1:
        print(
            f"unable to interpolate chain with 1 data point"
        )
        return 0
    else:
        if type == "call":
            cutoff = chain.iloc[-1]
            cutoffminus1 = chain.iloc[-2]
            c1 = cutoff["strike"]
            f1 = (cutoff["cbid"] + cutoff["cask"]) / 2
            x = cutoff["strike"] - cutoffminus1["strike"]
            y = (cutoff["cbid"] + cutoff["cask"]) / 2 - (
                cutoffminus1["cbid"] + cutoffminus1["cask"]
            ) / 2
            if round(y, 3) >= 0:
                f1 = (cutoffminus1["cbid"] + cutoffminus1["cask"]) / 2
                y = -f1
            if component == "fiv":
                tail = max(-(f1**2 *x )/(2 * c1**2 * y),0)
            elif component == "varq":
                tail = max(f1 - c1*f1 - (f1**2 * x)/(2 * y), 0)
            else:
                raise ValueError(f"invalid component {component}")
        elif type == "put":
            cutoff = chain.iloc[0]
            cutoffplus1 = chain.iloc[1]
            c1 = cutoff["strike"]
            f1 = (cutoff["pbid"] + cutoff["pask"]) / 2
            x = cutoff["strike"] - cutoffplus1["strike"]
            y = (cutoff["pbid"] + cutoff["pask"]) / 2 - (
                cutoffplus1["pbid"] + cutoffplus1["pask"]
            ) / 2
            if round(y, 3) >= 0:
                f1 = (cutoffplus1["pbid"] + cutoffplus1["pask"]) / 2
                y = f1
            if component == "fiv":
                tail = max((f1**2 *x )/(2 * c1**2 * y),0)
            elif component == "varq":
                tail = max(-(f1 - c1*f1 - (f1**2 * x)/(2 * y)), 0)
            else:
                raise ValueError(f"invalid component {component}")
        else:
            raise ValueError(f"invalid type {type}")
        # print(f'{dte} dte {type} adj tail {tail * 2/dte * np.exp(rfr * dte) * 10000} bound {cutoff[' [STRIKE]']} terms {(y/x) - (y*c1/x + y/2)/(c1 + x/2) - (y/(2*x)) * ((x/2 + f1 * (x/y))/(c1 + x/2))**2}, {2/dte}, {rfr} ')
        return tail

def batch_findoption(items, func, batch_size=90):
    # generator of batches
    batches = (
        items[i : i + batch_size]
        for i in range(0, len(items), batch_size)
    )
    return list(chain.from_iterable(func([option.symbol for option in batch]) for batch in batches))

def fetch_options(option_batch):
    optiondata = get_market_data_by_type(session, options = option_batch)
    return optiondata

def convertchain(chain):
    all_results = batch_findoption(chain, fetch_options, batch_size=90)

    res_map = {res.symbol: res for res in all_results}
    #TODO: this thing is fking updating during globex hours, thats why the prices will look all wrong
    combined = [
        {
            "symbol":       opt.symbol,
            "strike_price": float(opt.strike_price),
            "option_type":  opt.option_type,
            "bid":          float(getattr(res_map.get(opt.symbol, None), "bid",  np.nan)),
            "ask":          float(getattr(res_map.get(opt.symbol, None), "ask",  np.nan)),
        }
        for opt in chain
    ]

    df = pd.DataFrame(combined)
    vc = df["strike_price"].value_counts()
    # find any strikes that don’t have exactly 2 entries
    wrong = vc[vc != 2]

    # assert, printing out the offending strike_price values
    assert wrong.empty, f"Strikes without exactly two entries: {wrong.index.tolist()}"

    df["type"] = df["option_type"].apply(
        lambda ot: "C" if ot == OptionType.CALL
                   else "P" if ot == OptionType.PUT
                   else (_ for _ in ()).throw(ValueError(f"Unexpected OptionType: {ot!r}"))
    )

    df_pivot = (
        df
        .pivot(index="strike_price", columns="type", values=["bid", "ask"])
        .reset_index()
    )

    df_pivot.columns = [
        f"{col_type.lower()}{val}" if isinstance(col_type, str)
        else col_type  # this picks up the strike_price index as-is
        for val, col_type in df_pivot.columns
    ]

    # 7. Rename strike_price → strike, reorder
    dfchain = (
        df_pivot
        .rename(columns={"strike_price": "strike"})
        [["cbid", "cask", "strike", "pbid", "pask"]]
    )
    return dfchain

In [26]:
vx1expiryf = VXfm.last_trade_date
vx1expiryb = vx1expiryf + timedelta(days=30)
vx2expiryf = VXbm.last_trade_date
vx2expiryb = vx2expiryf + timedelta(days=30)
rfr = 4.232
today = date.today()
ervart1values = []
ervart2values = []
values = {
        "ervart1": pd.NA,
        "varqt1": pd.NA,
        "eqv1": pd.NA,
        "ervart2": pd.NA,
        "varqt2": pd.NA,
        "eqv2": pd.NA,
    }

spot = float(SPXdata[0].last)
VXstyle = "prev-close"
if VXstyle == "last":
    vx1close = float(VXfmdata[0].last)
    vx2close = float(VXbmdata[0].last)
elif VXstyle == "close":
    vx1close = float(VXfmdata[0].close)
    vx2close = float(VXbmdata[0].close)
elif VXstyle == "prev-close":
    vx1close = float(VXfmdata[0].prev_close)
    vx2close = float(VXbmdata[0].prev_close)
else:
    raise ValueError(f"invalid style {VXstyle}")

print('calc efg1')
for vx1expiry in [vx1expiryf, vx1expiryb]:
    t1 = (vx1expiry - today).days
    if vx1expiry in SPXexpiries:
        print('calc ervar:', vx1expiry)
        t1chain = SPXoptionchain[vx1expiry]

        ervart1 = calc_forward_iv(
            t1chain, t1, rfr, spot
        )
    else:
        if (
            SPXexpiries[0] > vx1expiry
        ):  # sometimes the closest vx futures expiry is before the closest spx option expiry, this happens in earlier years
            exp1 = today  # the workaround is to let the expiry be today to "simulate" a 0dte that expired at close today
        else:
            exp1 = exp1 = max([d for d in SPXexpiries if d < vx1expiry], default=None)
        exp2 = min([d for d in SPXexpiries if d > vx1expiry], default=None)
        print('calc ervar:', exp1, exp2)
        t1e1 = (exp1 - today).days  # front expiry of this vix future
        t1e2 = (
            exp2 - today
        ).days  # back expiry of this vix future, does NOT refer to the expiry of either the front or back timestamps in vx1expiryf or b
        if date == exp1:
            vart1e1 = 0
            # print("date = exp1 for vx1") #debug line
        else:
            t1e1chain = SPXoptionchain[exp1]
            vart1e1 = calc_forward_iv(
                t1e1chain, t1e1, rfr, spot
            )
        t1e2chain = SPXoptionchain[exp2]
        vart1e2 = calc_forward_iv(
            t1e2chain, t1e2, rfr, spot
        )
        ervart1 = ((t1e2 - t1) * t1e1 * vart1e1 + (t1 - t1e1) * t1e2 * vart1e2) / (
            (t1e2 - t1e1) * t1
        )  # https://gregorygundersen.com/blog/2023/09/10/deriving-vix/
    ervart1values.append((t1, ervart1))
ervart1 = (
    ervart1values[1][0] * ervart1values[1][1]
    - ervart1values[0][0] * ervart1values[0][1]
) / (
    ervart1values[1][0] - ervart1values[0][0]
)  # (Tb * vb - Ta * va) / (Tb - Ta)
values["ervart1"] = ervart1

assert vx1expiryf in VIXexpiries
t1vixchain = VIXoptionchain[vx1expiryf]
print('calc varq:', vx1expiryf)
varqt1 = calc_varq(t1vixchain, (vx1expiryf - today).days, rfr, vx1close)
values["varqt1"] = varqt1

if pd.notna(ervart1) and pd.notna(varqt1):
    values["eqv1"] = np.sqrt(ervart1 - varqt1)

print('calc efg2')
for vx2expiry in [vx2expiryf, vx2expiryb]:
    t2 = (vx2expiry - today).days
    if vx2expiry in SPXexpiries:
        print('calc ervar:', vx2expiry)
        t2chain = SPXoptionchain[vx2expiry]
        ervart2 = calc_forward_iv(
            t2chain, t2, rfr, spot
        )
    else:
        if (
            SPXexpiries[0] > vx2expiry
        ):
            exp1 = today
        else:
            exp1 = max([d for d in SPXexpiries if d < vx2expiry], default=None)
        exp2 = min([d for d in SPXexpiries if d > vx2expiry], default=None)
        print('calc ervar:', exp1, exp2)
        t2e1 = (exp1 - today).days  # front expiry of this vix future
        t2e2 = (
            exp2 - today
        ).days
        if date == exp1:
            vart2e1 = 0
        else:
            t2e1chain = SPXoptionchain[exp1]
            vart2e1 = calc_forward_iv(
                t2e1chain, t2e1, rfr, spot
            )
        t2e2chain = SPXoptionchain[exp2]
        vart2e2 = calc_forward_iv(
            t2e2chain, t2e2, rfr, spot
        )
        ervart2 = ((t2e2 - t2) * t2e1 * vart2e1 + (t2 - t2e1) * t2e2 * vart2e2) / (
            (t2e2 - t2e1) * t2
        )
    ervart2values.append((t2, ervart2))
ervart2 = (
    ervart2values[1][0] * ervart2values[1][1]
    - ervart2values[0][0] * ervart2values[0][1]
) / (
    ervart2values[1][0] - ervart2values[0][0]
)
values["ervart2"] = ervart2

assert vx2expiryf in VIXexpiries
t2vixchain = VIXoptionchain[vx2expiryf]
print('calc varq:', vx2expiryf)
varqt2 = calc_varq(t2vixchain, (vx2expiryf - today).days, rfr, vx2close)
values["varqt2"] = varqt2

if pd.notna(ervart2) and pd.notna(varqt2):
    values["eqv2"] = np.sqrt(ervart2 - varqt2)

efg1, efg2 = 100*(vx1close - values["eqv1"])/vx1close, 100*(vx2close - values["eqv2"])/vx2close
print(values)
print(f'SPX last price: {spot}')
print(f'VX1 prev close: {VXfmdata[0].prev_close}, VX2 prev close: {VXbmdata[0].prev_close}')
print(f'VX1 close: {VXfmdata[0].close}, VX2 close: {VXbmdata[0].close}')
print(f'VX1 last price: {VXfmdata[0].last}, VX2 last price: {VXbmdata[0].last}')
print(f'EFG1: {efg1}, EFG2: {efg2}')

calc efg1
calc ervar: 2025-06-18
calc ervar: 2025-07-18
calc varq: 2025-06-18
calc efg2
calc ervar: 2025-08-15 2025-08-29
calc ervar: 2025-09-19
calc varq: 2025-08-20
{'ervart1': np.float64(477.95445114172145), 'varqt1': np.float64(63.744372861744694), 'eqv1': np.float64(20.352151686737614), 'ervart2': np.float64(594.7737864126643), 'varqt2': np.float64(182.42023779983268), 'eqv2': np.float64(20.306490307604406)}
SPX last price: 5842.01
VX1 prev close: 20.55, VX2 prev close: 21.35
VX1 close: None, VX2 close: None
VX1 last price: 20.55, VX2 last price: 21.45
EFG1: 0.9627655146588183, EFG2: 4.8876332196515


In [28]:
fetch_options(SPXoptionchain[SPXexpiries[2]][300].symbol)

[MarketData(symbol='SPXW  250527C05860000' instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'> updated_at=datetime.datetime(2025, 5, 23, 2, 14, 48, 903000, tzinfo=TzInfo(UTC)) bid_size=Decimal('14.0') ask_size=Decimal('14.0') mark=Decimal('29.0') close_price_type=<ClosePriceType.REGULAR: 'Regular'> summary_date=datetime.date(2025, 5, 23) prev_close_date=datetime.date(2025, 5, 22) prev_close_price_type=<ClosePriceType.REGULAR: 'Regular'> halt_start_time=-1 halt_end_time=-1 ask=Decimal('29.3') bid=Decimal('28.7') day_high_price=Decimal('45.19') day_low_price=Decimal('26.3') last=Decimal('29.15') last_mkt=Decimal('29.15') mid=Decimal('29.0') open=Decimal('32.96') prev_close=Decimal('29.15') volume=Decimal('552.0'))]

In [32]:
convertchain(VIXoptionchain[datetime.strptime("2025-08-20", "%Y-%m-%d").date()])

,cbid,cask,strike,pbid,pask
0,11.15,11.40,10.0,0.00,0.06
1,10.65,10.90,10.5,0.00,0.06
2,10.15,10.40,11.0,0.00,0.06
3,9.70,9.95,11.5,0.00,0.07
4,9.20,9.45,12.0,0.00,0.07
5,8.70,8.95,12.5,0.00,0.08
6,8.20,8.45,13.0,0.00,0.10
7,7.75,8.00,13.5,0.00,0.10
8,7.30,7.55,14.0,0.07,0.16
9,6.85,7.10,14.5,0.07,0.22


In [30]:
d1 = datetime.strptime("2025-08-29", "%Y-%m-%d").date()
d2 = datetime.strptime("2025-09-19", "%Y-%m-%d").date()
print(d1, (d1-today).days)
print(d2, (d2-today).days)
erv1 = calc_forward_iv(SPXoptionchain[d1], (d1-today).days, rfr, spot)
erv2 = calc_forward_iv(SPXoptionchain[d2], (d2-today).days, rfr, spot)
print(erv1, erv2)
print((erv2*(d2-today).days - erv1*(d1-today).days) / ((d2-today).days-(d1-today).days))

2025-08-29 98
2025-09-19 119
508.9771464165523 521.5631159684141
580.2976405437691


In [6]:
VXfm

Future(instrument_type=<InstrumentType.FUTURE: 'Future'> symbol='/VXQ25' product_code='VX' tick_size=Decimal('0.05') notional_multiplier=Decimal('1000.0') display_factor=Decimal('1.0') last_trade_date=datetime.date(2025, 8, 20) expiration_date=datetime.date(2025, 8, 20) closing_only_date=datetime.date(2025, 8, 20) active=True stops_trading_at=datetime.datetime(2025, 8, 20, 13, 30, tzinfo=TzInfo(UTC)) expires_at=datetime.datetime(2025, 8, 20, 13, 30, tzinfo=TzInfo(UTC)) product_group='CFE' exchange='CFE' streamer_exchange_code='XCBF' back_month_first_calendar_symbol=True streamer_symbol='/VXQ25:XCBF' is_tradeable=True future_product=FutureProduct(root_symbol='/VX' code='VX' description='Volatility Index' exchange='CFE' product_type='Financial' listed_months=[<FutureMonthCode.JAN: 'F'>, <FutureMonthCode.FEB: 'G'>, <FutureMonthCode.MAR: 'H'>, <FutureMonthCode.APR: 'J'>, <FutureMonthCode.MAY: 'K'>, <FutureMonthCode.JUN: 'M'>, <FutureMonthCode.JUL: 'N'>, <FutureMonthCode.AUG: 'Q'>, <FutureM

In [5]:
VXfutures

[Future(instrument_type=<InstrumentType.FUTURE: 'Future'> symbol='/VXN25' product_code='VX' tick_size=Decimal('0.05') notional_multiplier=Decimal('1000.0') display_factor=Decimal('1.0') last_trade_date=datetime.date(2025, 7, 16) expiration_date=datetime.date(2025, 7, 16) closing_only_date=datetime.date(2025, 7, 16) active=True next_active_month=True stops_trading_at=datetime.datetime(2025, 7, 16, 13, 30, tzinfo=TzInfo(UTC)) expires_at=datetime.datetime(2025, 7, 16, 13, 30, tzinfo=TzInfo(UTC)) product_group='CFE' exchange='CFE' streamer_exchange_code='XCBF' back_month_first_calendar_symbol=True streamer_symbol='/VXN25:XCBF' is_tradeable=True future_product=FutureProduct(root_symbol='/VX' code='VX' description='Volatility Index' exchange='CFE' product_type='Financial' listed_months=[<FutureMonthCode.JAN: 'F'>, <FutureMonthCode.FEB: 'G'>, <FutureMonthCode.MAR: 'H'>, <FutureMonthCode.APR: 'J'>, <FutureMonthCode.MAY: 'K'>, <FutureMonthCode.JUN: 'M'>, <FutureMonthCode.JUL: 'N'>, <FutureMonth

In [ ]:
VXfmdata